#Thông tin nhóm
STT: 8

1712704 - Nguyễn Phước Quang.

1712892 - Nguyễn Thị Tuyết.



# Đồ án cuối kỳ môn nhập môn khoa học dữ liệu

# 1.Thu thập dữ liệu

In [1]:
# import cac thu vien can thiet
import requests
import csv
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
import re
import time as t


In [2]:
per_page=1
url = f'https://www.imdb.com/search/title/?title_type=feature&amp&release_date=1995-10-10,2020-12-12&amp&start={per_page}&amp&ref_=adv_nxt'
html_text = requests.get(url).text
tree= BeautifulSoup(html_text, 'html.parser')

In [3]:
films = tree.find_all('div', {'class': 'lister-item-content'})
test=tree.find_all('div',class_="desc")


1

In [6]:
# các biến dùng để lưu các thuộc tính của bộ phim
list_title = []
duration = []
category = []
scores = []
score_voting=[]
amount_vote=[]
amount_gross=[]
check=True
i=1
while(i<10000): 
  url = f'https://www.imdb.com/search/title/?title_type=feature&amp&release_date=1995-10-10,2020-12-12&amp&start={i}&amp&ref_=adv_nxt'
  html_text = requests.get(url).text
  tree= BeautifulSoup(html_text, 'html.parser')
  films = tree.find_all('div', {'class': 'lister-item-content'})
  for film in films:

    # lay ten film
    temp=film
    title=temp.find('a').get_text().replace('\n','').replace('\t','')
    #print(title)
    list_title.append(title)

    try:
      # lay  thoi gian cua film
      head=temp.find_all(class_ = "text-muted")
      time=head[1]
      runtime = time.find(class_="runtime").get_text()
      runtime = re.findall("[0-9]+", runtime)
      runtime=int(runtime[0])
      duration.append(runtime)
      #print(runtime)
    except:
      duration.append(np.nan)
    # lay the loai film
    
    try:
      genre = time.find(class_= "genre").get_text()
      genre=genre.lstrip("\n").rstrip("\n").replace('\n','').replace('\t','')
      #print(genre)
      category.append(genre)
    except:
      category.append(np.nan)

    try: 
      # lay diem  danh gia vote
      voting=temp.find(class_="inline-block ratings-imdb-rating").get_text()

      voting=float(voting.lstrip("\n"))
      #print(voting)
      score_voting.append(voting)
    except:
      score_voting.append(np.nan)
    # lay diem Metascore

    try: 
      meta_score=temp.find(class_="inline-block ratings-metascore").get_text()
      meta_score =re.findall("[0-9]+", meta_score)
      meta_score=(int)(meta_score[0])
      #print(meta_score)
      scores.append(meta_score)
    except:
      scores.append(np.nan)
    # lay number vote
    try:
      number_votes=temp.find(class_="sort-num_votes-visible").find_all("span")[1].get_text()
      number_votes=(int)(number_votes.replace(',',''))
      amount_vote.append(number_votes)
      #print(number_votes)
    except:
      amount_vote.append(np.nan)
    # lay danh thu  film
    try:
      gross=temp.find(class_="sort-num_votes-visible").find_all("span")[4].get_text()
      gross=gross.lstrip("$").rstrip("M")
      amount_gross.append(gross)
      #print((float)(gross))
    except:
      amount_gross.append(np.nan)

  # kiem tra trang ke tiep
  i=i+50
  #print(i)
  t.sleep(1)
  
 
# viết dữ liệu xuống file csv
data=[list_title,duration,category,scores,score_voting,amount_vote,amount_gross]
a=np.array(data)
dataset = pd.DataFrame({'Title': list_title, 'Category': category,'Duration':duration,'Socres':scores,'Score_voting':score_voting,'Amount_vote':amount_vote,'Amount_gross':amount_gross})
dataset.to_csv("a.csv",sep=',')
